# This is a student solution in TDT05 2020
## The notebook is read-only because the data files are not available

In [1]:
# Import and init h2o lib. Running on java
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.10" 2021-01-19; OpenJDK Runtime Environment AdoptOpenJDK (build 11.0.10+9); OpenJDK 64-Bit Server VM AdoptOpenJDK (build 11.0.10+9, mixed mode)
  Starting server from /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/y1/6h3k81fn12g2nt2tpwdrrkn40000gn/T/tmp1bq6mvfl
  JVM stdout: /var/folders/y1/6h3k81fn12g2nt2tpwdrrkn40000gn/T/tmp1bq6mvfl/h2o_adrianlangseth_started_from_python.out
  JVM stderr: /var/folders/y1/6h3k81fn12g2nt2tpwdrrkn40000gn/T/tmp1bq6mvfl/h2o_adrianlangseth_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,16 days
H2O_cluster_name:,H2O_from_python_adrianlangseth_2h9dt4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [3]:
# Reading csv to h2o dataframe file
df = h2o.import_file('../tdt05-2021-challenge-2/challenge2_train.csv')
df_test = h2o.import_file('../tdt05-2021-challenge-2/challenge2_test.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [4]:
# Reading csv to h2o dataframe file
train = pd.read_csv('../tdt05-2021-challenge-2/challenge2_train.csv')
test = pd.read_csv('../tdt05-2021-challenge-2/challenge2_test.csv')

ord_features = ['f1', 'f2', 'f3', 'f5', 'f7', 'f10', 'f13', 'f18', 'f19', 'f27']
num_ords = ['f3', 'f5', 'f7', 'f19', 'f27']
alpha_ords = ['f1', 'f2', 'f10', 'f18']

numeric = ['f11', 'f17', 'f24', 'f28']
bell_curve = ['f11', 'f28']
long_tail = ['f17', 'f24']  # F24 has -1 as null values. Remove before scaling

bin_features = ['f0', 'f4', 'f6', 'f25', 'f26']
cyc_features = ['f16', 'f21']
nom_features = ['f8', 'f9', 'f12', 'f14', 'f15', 'f22', 'f23']  # hexes
duplicate = ['f20']

all_feat=bin_features+nom_features+ord_features+['f16_sin', 'f16_cos', 'f21_sin', 'f21_cos']+numeric

categorical = bin_features+nom_features+ord_features+ ['f1_0', 'f1_1']+['f16_sin', 'f16_cos', 'f21_sin', 'f21_cos']

#Split up the dub
train['f1_0'] = train['f1'].apply(lambda x: x[0] if type(x) is str else x)
test['f1_0'] = test['f1'].apply(lambda x: x[0] if type(x) is str else x)
train['f1_1'] = train['f1'].apply(lambda x: x[1] if type(x) is str else x)
test['f1_1'] = test['f1'].apply(lambda x: x[1] if type(x) is str else x)
train.drop(['f1'], axis=1, inplace=True)
test.drop(['f1'], axis=1, inplace=True)
ord_features.remove('f1')
ord_features.extend(['f1_0', 'f1_1'])

# EHM something

train[['f5']].replace(-1.0, np.nan, inplace=True)
test[['f5']].replace(-1.0, np.nan, inplace=True)

# Convert those alphas which are very indicative

train['f13'] = train['f13'].apply(lambda x: (ord(x) - ord('a'))/14 if pd.notnull(x) else x)
test['f13'] = test['f13'].apply(lambda x: (ord(x) - ord('a'))/14 if pd.notnull(x) else x)
ord_features.remove('f13')

train['f10'] = train['f10'].apply(lambda x: (ord(x) - ord('A'))/25 if pd.notnull(x) else x)
test['f10'] = test['f10'].apply(lambda x: (ord(x) - ord('A'))/25 if pd.notnull(x) else x)
ord_features.remove('f10')

train['f3'] = train['f3'].apply(lambda x: float(x)/3 if pd.notnull(x) else x)
test['f3'] = test['f3'].apply(lambda x: float(x)/3 if pd.notnull(x) else x)
ord_features.remove('f3')

## ChooChoo here comes the number soup
scl = preprocessing.StandardScaler()
train[['f11', 'f28']] = scl.fit_transform(train[['f11', 'f28']])
test[['f11', 'f28']] = scl.transform(test[['f11', 'f28']])
    
train[['f24']].replace(-1.0, np.nan, inplace=True)
test[['f24']].replace(-1.0, np.nan, inplace=True)
train[['f17', 'f24']] = np.log(train[['f17', 'f24']])
test[['f17', 'f24']] = np.log(test[['f17', 'f24']])

scl = preprocessing.StandardScaler()
train[['f17', 'f24']] = scl.fit_transform(train[['f17', 'f24']])
test[['f17', 'f24']] = scl.transform(test[['f17', 'f24']])

# 9 million bicycles in Beijing

train['f16_sin'] = np.sin((train['f16'] - 1) * (2. * np.pi / 12))
train['f16_cos'] = np.cos((train['f16'] - 1) * (2. * np.pi / 12))
test['f16_sin'] = np.sin((test['f16'] - 1) * (2. * np.pi / 12))
test['f16_cos'] = np.cos((test['f16'] - 1) * (2. * np.pi / 12))
train.drop(['f16'], axis=1, inplace=True)
test.drop(['f16'], axis=1, inplace=True)

train['f21_sin'] = np.sin((train['f21'] - 1) * (2. * np.pi / 7))
train['f21_cos'] = np.cos((train['f21'] - 1) * (2. * np.pi / 7))
test['f21_sin'] = np.sin((test['f21'] - 1) * (2. * np.pi / 7))
test['f21_cos'] = np.cos((test['f21'] - 1) * (2. * np.pi / 7))
train.drop(['f21'], axis=1, inplace=True)
test.drop(['f21'], axis=1, inplace=True)
    
train['f16_sin'].fillna(0, inplace=True)
train['f16_cos'].fillna(0, inplace=True)
test['f16_sin'].fillna(0, inplace=True)
test['f16_cos'].fillna(0, inplace=True)

train['f21_sin'].fillna(0, inplace=True)
train['f21_cos'].fillna(0, inplace=True)
test['f21_sin'].fillna(0, inplace=True)
test['f21_cos'].fillna(0, inplace=True)

train['f19'] = train['f19'].apply(lambda x: int(x*10) if pd.notnull(x) else x)
test['f19'] = test['f19'].apply(lambda x: int(x*10) if pd.notnull(x) else x)

target = train['target']
test_id = test['id']

train.drop(['id', 'f20'], axis=1, inplace=True)
test.drop(['id', 'f20'], axis=1, inplace=True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in log
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encount

In [5]:
categorical = bin_features+nom_features+ord_features # + ['f1_0', 'f1_1']# +['f16_sin', 'f16_cos', 'f21_sin', 'f21_cos']
categorical.sort()
print(categorical)

['f0', 'f12', 'f14', 'f15', 'f18', 'f19', 'f1_0', 'f1_1', 'f2', 'f22', 'f23', 'f25', 'f26', 'f27', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9']


In [6]:
train['f5'].unique()

array([nan,  7., 10., 11.,  6.,  5.,  4.,  8.,  3.,  9.,  0.,  2.,  1.,
       -1.])

In [7]:
train[categorical].head().T

,0,1,2,3,4
f0,1,0,NaN,1,1
f12,1c756c04a,5d1ac7760,f14f5e4a5,56d35c774,5622ee17f
f14,168e51823,558613041,ae14d280b,86ccc2ee4,3162d6e8b
f15,7861df0a8,1d88b0a79,b72c0bbc2,a5eef5d8b,7861df0a8
f18,F,F,C,D,D
f19,5,4,5,6,5
f1_0,g,R,I,r,p
f1_1,L,j,n,A,E
f2,e,c,a,c,c
f22,R,NaN,G,G,B


In [8]:
train.f13.unique()
cats = ['target', 'f0', 'f2', 'f3', 'f4', 'f4', 'f4' ]

In [9]:
h_train = h2o.H2OFrame(train)
h_test = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [10]:
for col in categorical:
    h_train[col] = h_train[col].asfactor()
    h_test[col] = h_test[col].asfactor()


h_train['target'] = h_train['target'].asfactor()

In [11]:
# Removing labels from train set
y = "target"
x = h_train.columns
x.remove(y)

In [12]:
# Running fit/training on train set with cross validation n=5
aml = H2OAutoML(max_models = 25, seed = 42)
aml.train(x = x, y = y, training_frame = h_train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_7_AutoML_1_20211023_142742

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.09112794924135313
RMSE: 0.3018740618889823
LogLoss: 0.29860612707845063
Null degrees of freedom: 9963
Residual degrees of freedom: 9946
Null deviance: 9660.898713412416
Residual deviance: 5950.622900419364
AIC: 5986.622900419364
AUC: 0.915069165275142
AUCPR: 0.7571265375680483
Gini: 0.8301383305502841

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.306737881548967: 


,,0,1,Error,Rate
0,0,7338.0,743.0,0.0919,(743.0/8081.0)
1,1,505.0,1378.0,0.2682,(505.0/1883.0)
2,Total,7843.0,2121.0,0.1253,(1248.0/9964.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.306738,0.688312,182.0
1,max f2,0.218523,0.753800,229.0
2,max f0point5,0.412121,0.720696,136.0
3,max accuracy,0.410131,0.886190,137.0
4,max precision,0.884198,1.000000,0.0
5,max recall,0.028460,1.000000,370.0
6,max specificity,0.884198,1.000000,0.0
7,max absolute_mcc,0.321586,0.612021,175.0
8,max min_per_class_accuracy,0.229848,0.833776,222.0
9,max mean_per_class_accuracy,0.228608,0.834606,223.0



Gains/Lift Table: Avg response rate: 18,90 %, avg score: 18,35 %


...

,,0,1,Error,Rate
0,0,30347.0,10397.0,0.2552,(10397.0/40744.0)
1,1,3253.0,6003.0,0.3514,(3253.0/9256.0)
2,Total,33600.0,16400.0,0.273,(13650.0/50000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.218641,0.467961,222.0
1,max f2,0.130045,0.609583,288.0
2,max f0point5,0.349876,0.444747,143.0
3,max accuracy,0.515038,0.822740,70.0
4,max precision,0.832049,1.000000,0.0
5,max recall,0.007142,1.000000,397.0
6,max specificity,0.832049,1.000000,0.0
7,max absolute_mcc,0.250425,0.326111,200.0
8,max min_per_class_accuracy,0.194564,0.697551,239.0
9,max mean_per_class_accuracy,0.184074,0.700950,246.0



Gains/Lift Table: Avg response rate: 18,51 %, avg score: 18,51 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.01,0.632443,3.759723,3.759723,0.6960,0.687469,0.696000,0.687469,0.037597,0.037597,275.972342,275.972342,0.033867
1,2,0.02,0.577029,3.230337,3.495030,0.5980,0.600674,0.647000,0.644071,0.032303,0.069901,223.033708,249.503025,0.061237
2,3,0.03,0.539807,3.089888,3.359983,0.5720,0.557537,0.622000,0.615227,0.030899,0.100799,208.988764,235.998271,0.086883
3,4,0.04,0.512113,2.819793,3.224935,0.5220,0.525945,0.597000,0.592906,0.028198,0.128997,181.979257,222.493518,0.109215
4,5,0.05,0.487498,2.646932,3.109334,0.4900,0.499182,0.575600,0.574161,0.026469,0.155467,164.693172,210.933449,0.129426
5,6,0.10,0.400159,2.374676,2.742005,0.4396,0.440181,0.507600,0.507171,0.118734,0.274201,137.467589,174.200519,0.213774
6,7,0.15,0.343450,2.033276,2.505762,0.3764,0.370667,0.463867,0.461670,0.101664,0.375864,103.327571,150.576203,0.277175
7,8,0.20,0.299870,1.782627,2.324978,0.3300,0.320921,0.430400,0.426483,0.089131,0.464996,78.262748,132.497839,0.325196
8,9,0.30,0.232978,1.463915,2.037957,0.2710,0.264601,0.377267,0.372522,0.146392,0.611387,46.391530,103.795736,0.382126
9,10,0.40,0.183160,1.163570,1.819360,0.2154,0.206963,0.336800,0.331132,0.116357,0.727744,16.356958,81.936041,0.402199


In [13]:
lb = aml.leaderboard

In [14]:
# Showing best perforing models
lb.head()

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_7_AutoML_1_20211023_142742,0.768375,0.406531,0.42976,0.303313,0.358422,0.128466
StackedEnsemble_AllModels_3_AutoML_1_20211023_142742,0.768323,0.40655,0.42936,0.30868,0.358485,0.128511
StackedEnsemble_AllModels_4_AutoML_1_20211023_142742,0.76828,0.406557,0.429664,0.308764,0.35843,0.128472
StackedEnsemble_BestOfFamily_4_AutoML_1_20211023_142742,0.766297,0.407637,0.427669,0.306432,0.358886,0.128799
StackedEnsemble_AllModels_2_AutoML_1_20211023_142742,0.766206,0.407833,0.426635,0.310696,0.359017,0.128893
StackedEnsemble_BestOfFamily_7_AutoML_1_20211023_142742,0.766131,0.407782,0.427282,0.308969,0.35893,0.128831
StackedEnsemble_BestOfFamily_3_AutoML_1_20211023_142742,0.766096,0.407805,0.426914,0.310044,0.358984,0.128869
StackedEnsemble_AllModels_6_AutoML_1_20211023_142742,0.764646,0.409175,0.420699,0.303019,0.359686,0.129374
StackedEnsemble_AllModels_1_AutoML_1_20211023_142742,0.764608,0.408774,0.424477,0.312267,0.359413,0.129178
StackedEnsemble_BestOfFamily_2_AutoML_1_20211023_142742,0.764605,0.408817,0.424339,0.312303,0.359437,0.129195


In [15]:
preds = aml.predict(h_test)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'f8' has levels not trained on: ["072188131", "0a7e014d0", "1065f10dd", "1249f6c43", "2394a46de", "34d9d24e8", "34f96fad9", "3ac2f6841", "432e3fc6a", "5fb04046c", ...6 not listed..., "d1e6704ed", "d2eeb11ad", "d6017b5e1", "d61dbecb0", "e090cd246", "e21de3f04", "e45f9067a", "ea68ecb01", "f81531e13", "ff12eee03"]
  warnings.warn(w)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'f14' has levels not trained on: ["44a2e6186", "53554f740", "673d2edda", "7335087fd", "c6c90443c", "d6bb2181a", "eb59160cf"]
  warnings.warn(w)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'f23' has levels not trained on: ["03a6514a0", "0850be70d", "0ab3b52a4", "116ada84e", "1a3496903", "273fb836b", "

In [16]:
preds_list = h2o.as_list(preds)

In [17]:
p1 = preds_list['p1']


In [18]:
path='../tdt05-2021-challenge-2/'
submission=pd.read_csv(path+'sample_submission.csv')

In [19]:
submission['target'] =p1 #preds
submission.to_csv('results/automlv2.csv', index=None)
submission.head()

,target,id
0,0.311217,50000
1,0.437817,50001
2,0.135168,50002
3,0.452812,50003
4,0.133838,50004


In [20]:
submission=pd.read_csv(path+'sample_submission.csv')
own = pd.read_csv('results/all_feats_13_and_11_asnum_hyp_cat_cyclicals_nans_fixed_7_folds.csv')
aml = pd.read_csv('results/automlv2.csv')

In [21]:
submission['target'] = (aml.target*0.76613 + own.target*0.76956)/(.76613+0.76956)

In [22]:
submission.to_csv('results/new_aml_blend.csv', index=None)